In [3]:
import gc
import pandas as pd
import numpy as np
import os
# import xgboost as xgb
import lightgbm as lgb
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score  
from sklearn.metrics import confusion_matrix  
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.model_selection import ParameterGrid

In [4]:
####################训练集#######################
DATA_PATH = "datasets"
csv_path = os.path.join(DATA_PATH, "atec_anti_fraud_train.csv")
data = pd.read_csv(csv_path)

In [5]:
data.loc[data['label'] == -1, 'label'] = 1
# data_to_label = data[data['label'] == -1]
# data = data[data.label.isin([0, 1])]

In [6]:
data.loc[data['label'] == 1, 'label'] = 2
data.loc[data['label'] == 0, 'label'] = 1
data.loc[data['label'] == 2, 'label'] = 0

In [9]:
# data.sort_values("date",inplace=True)
# data.interpolate(method="nearest", inplace=True)
# data.sort_index(inplace=True)

In [7]:
feature_nan = data.isnull().sum()
drop_list = feature_nan[feature_nan / data.shape[0] > 0.5].reset_index()['index'].tolist()

In [8]:
print(drop_list)

['f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47']


In [11]:
data = data.drop(drop_list, axis=1)

In [12]:
####################测试集#######################
csv_path = os.path.join(DATA_PATH, "atec_anti_fraud_test_b.csv")
data_test = pd.read_csv(csv_path)

In [13]:
data_test.sort_values("date",inplace=True)
data_test.interpolate(method="nearest", inplace=True) 
data_test.sort_index(inplace=True)

In [14]:
data_test = data_test.drop(drop_list, axis=1)

In [73]:
#data_test['day'] = data_test['date'].apply(lambda x: int(str(x)[6:8]))
# data.fillna(0, inplace=True)
# data_test.fillna(0, inplace=True)

In [15]:
data_train = data[data['date'] <= 20171021]
data_val = data[data['date'] > 20171021]
print(data.shape)
print(data_train.shape)
print(data_val.shape)

(994731, 300)
(749690, 300)
(245041, 300)


In [16]:
X_train = data_train.loc[:,'f1':].as_matrix()
y_train = data_train.loc[:,'label'].as_matrix()

X_val = data_val.loc[:,'f1':].as_matrix()
y_val = data_val.loc[:,'label'].as_matrix()

X_test = data_test.loc[:,'f1':].as_matrix()

In [17]:
#X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)
print(X_test.shape)

(749690, 297)
(749690,)
(245041, 297)
(245041,)
(500538, 297)


In [18]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train)

In [19]:
def eval_func(y_pred, y_true):
    fpr, tpr, thresholds = metrics.roc_curve(y_pred, y_true, pos_label=1)
    score = 0.4 * tpr[np.argwhere(fpr < 0.001)[-1]] + 0.3 * tpr[np.argwhere(
        fpr < 0.005)[-1]] + 0.3 * tpr[np.argwhere(fpr < 0.01)[-1]]
    return 'score', score, True

In [20]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'auc'},
    'num_leaves': 50,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'random_state': 0,
    'n_jobs': 8,
    'verbose': 0
}

In [21]:
model = lgb.train(params,
                lgb_train,
                num_boost_round=300,
                valid_sets=[lgb_eval],
                categorical_feature=[1,2,3,4,6,7,8,9,10,11,12,13,14,15,16,17,18,19],
#                 feval='auc',
                verbose_eval=True,
                early_stopping_rounds=20)

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:1038: UserWarning: categorical_feature in Dataset is overrided. New categorical_feature is [1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
  warnings.warn('categorical_feature in Dataset is overrided. New categorical_feature is {}'.format(sorted(list(categorical_feature))))
C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:681: UserWarning: categorical_feature in param dict is overrided.
  warnings.warn('categorical_feature in param dict is overrided.')


[1]	valid_0's auc: 0.892182
Training until validation scores don't improve for 20 rounds.
[2]	valid_0's auc: 0.925331
[3]	valid_0's auc: 0.928663
[4]	valid_0's auc: 0.929544
[5]	valid_0's auc: 0.931719
[6]	valid_0's auc: 0.931831
[7]	valid_0's auc: 0.932195
[8]	valid_0's auc: 0.932253
[9]	valid_0's auc: 0.932279
[10]	valid_0's auc: 0.932267
[11]	valid_0's auc: 0.932521
[12]	valid_0's auc: 0.953334
[13]	valid_0's auc: 0.954102
[14]	valid_0's auc: 0.954586
[15]	valid_0's auc: 0.954885
[16]	valid_0's auc: 0.955521
[17]	valid_0's auc: 0.955937
[18]	valid_0's auc: 0.956306
[19]	valid_0's auc: 0.956417
[20]	valid_0's auc: 0.957917
[21]	valid_0's auc: 0.958586
[22]	valid_0's auc: 0.958846
[23]	valid_0's auc: 0.95933
[24]	valid_0's auc: 0.959595
[25]	valid_0's auc: 0.959784
[26]	valid_0's auc: 0.960068
[27]	valid_0's auc: 0.960786
[28]	valid_0's auc: 0.962616
[29]	valid_0's auc: 0.962799
[30]	valid_0's auc: 0.962837
[31]	valid_0's auc: 0.963954
[32]	valid_0's auc: 0.964056
[33]	valid_0's auc: 

In [22]:
print(params)
print(eval_func(1-y_train, 1 - model.predict(X_train, num_iteration=model.best_iteration)))
print(eval_func(1-y_val, 1 - model.predict(X_val, num_iteration=model.best_iteration)))

{'task': 'train', 'boosting_type': 'gbdt', 'objective': 'binary', 'metric': {'auc'}, 'num_leaves': 50, 'learning_rate': 0.05, 'feature_fraction': 0.9, 'bagging_fraction': 0.8, 'bagging_freq': 5, 'random_state': 0, 'n_jobs': 8, 'verbose': 0, 'categorical_column': [1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]}
('score', array([0.53041449]), True)
('score', array([0.41151011]), True)


In [23]:
y_pred = 1 - model.predict(X_test)
result = pd.DataFrame({'id':data_test['id'].as_matrix(), 'score':y_pred})
result.to_csv("submission.csv", index=False)